# Predicting Services from Diagnoses

In [33]:
#Import libraries
from catboost import CatBoostClassifier # For the classifier 
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from tqdm import tqdm
from contextlib import contextmanager
import joblib
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, recall_score,  average_precision_score, make_scorer,  precision_recall_curve, auc
from xgboost import XGBClassifier
import numpy as np

import os
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules



import warnings
warnings.filterwarnings("ignore") # Suppress warnings to clean output


from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.multiclass import OneVsRestClassifier


In [34]:
# Read in our pre-split training and validation sets.
# The dataset looks like this: Each member has their own row, with an ID, ~300 diagnosis codes, and ~300 service codes
train_df = pd.read_csv(r"K:\Mktcare\DATA_INTEGRATION_AND_ANALYSIS\Ben\Hackathon\FormattedData\FilteredMasked\AllRecords\ALL_MMI_train.csv")
val_df = pd.read_csv(r"K:\Mktcare\DATA_INTEGRATION_AND_ANALYSIS\Ben\Hackathon\FormattedData\FilteredMasked\AllRecords\ALL_MMI_validation.csv")

In [35]:


# Identify diagnosis and service columns
diagnosis_cols = [col for col in train_df.columns if col.startswith('Diagnosis')]
service_cols = [col for col in train_df.columns if col.startswith('Service')]

# Combine diagnosis and service columns into sets per row
train_df['all_diagnoses'] = train_df[diagnosis_cols].apply(lambda row: set(row.dropna()), axis=1)
val_df['all_diagnoses'] = val_df[diagnosis_cols].apply(lambda row: set(row.dropna()), axis=1)

# Combine service columns into sets of strings per row
train_df['all_services'] = train_df[service_cols].apply(lambda row: set(row.dropna().astype(str)), axis=1)
val_df['all_services'] = val_df[service_cols].apply(lambda row: set(row.dropna().astype(str)), axis=1)


# Fit MultiLabelBinarizer on training diagnoses
mlb_diagnosis = MultiLabelBinarizer()
X_train = pd.DataFrame(mlb_diagnosis.fit_transform(train_df['all_diagnoses']),
                       columns=[f'Diagnosis_{code}' for code in mlb_diagnosis.classes_],
                       index=train_df.index)

X_val = pd.DataFrame(mlb_diagnosis.transform(val_df['all_diagnoses']),
                     columns=[f'Diagnosis_{code}' for code in mlb_diagnosis.classes_],
                     index=val_df.index)

# Fit MultiLabelBinarizer on training services
mlb_service = MultiLabelBinarizer()
y_train = pd.DataFrame(mlb_service.fit_transform(train_df['all_services']),
                       columns=[f'Service_{code}' for code in mlb_service.classes_],
                       index=train_df.index)

y_val = pd.DataFrame(mlb_service.transform(val_df['all_services']),
                     columns=[f'Service_{code}' for code in mlb_service.classes_],
                     index=val_df.index)
# Align X_val to X_train's columns
X_val = X_val.reindex(columns=X_train.columns, fill_value=0)

# Align y_val to y_train's columns
y_val = y_val.reindex(columns=y_train.columns, fill_value=0)


In [36]:


# Read column names from a text file and format them with a prefix
# This long_term_codes file contains a list of diagnosis codes that are considered to be long-term. These are the ones we can use to predict services
with open('long_term_codes.txt', 'r') as file:
    valid_columns = [
        "Diagnosis_" + line.strip().replace(" ", "_")
        for line in file if line.strip()
    ]



X_train_filtered = X_train[valid_columns]
X_val_filtered = X_val[valid_columns]

# Display the shape of the filtered datasets
print("Filtered X_train shape:", X_train_filtered.shape)
print("Filtered X_val shape:", X_val_filtered.shape)



Filtered X_train shape: (14209, 421)
Filtered X_val shape: (3552, 421)


In [37]:



# Assuming y_train is already defined as a DataFrame
# Define the mapping of categories to their associated CPT code columns to group service codes. These are the 10 services we are looking to predict 
# We found these 10 services after manually looking at the data and discussing with experts in the field

category_mapping = {
    'Service_Colonoscopy': ['Service_45380', 'Service_45378', 'Service_45385'],
    'Service_EGD': ['Service_43239'],
    'Service_Gallbladder_surgery': ['Service_47562'],
    'Service_CT_scan': ['Service_74177', 'Service_74176', 'Service_74178'],
    'Service_Infusions_drugs': [
        'Service_96372', 'Service_96374', 'Service_96375', 'Service_96361',
        'Service_96365', 'Service_96360', 'Service_96376', 'Service_96366',
        'Service_J0702', 'Service_J3490', 'Service_J3301',
        'Service_J0690', 'Service_J1170', 'Service_J2270', 'Service_J2001'
    ],
    'Service_Fentanyl':['Service_J3010'],
    # 'Service_Ceftriaxone':['Service_J0696'],
    'Service_Orthopedic_surgery': ['Service_20610', 'Service_20550', 'Service_29125', 'Service_29075'],
    'Service_ENT_Respiratory_surgery': ['Service_31231', 'Service_31575'],
    'Service_Electrocardiogram': ['Service_93000', 'Service_93010', 'Service_93005', 'Service_93017', 'Service_93018', 'Service_93015', 'Service_93016'],
    'Service_Hospital_Observation': ['Service_99238', 'Service_99232', 'Service_99239', 'Service_99223', 'Service_99233', 'Service_99222', 'Service_99231', 'Service_99221', 'Service_99217', 'Service_99220']
}

# Create a new DataFrame with category columns
category_df = pd.DataFrame()
category_df_val = pd.DataFrame()
# For each category, set the value to 1 if any associated column has a 1
for category, columns in category_mapping.items():
    category_df[category] = (y_train[columns].sum(axis=1) > 0).astype(int)
    category_df_val[category] = (y_val[columns].sum(axis=1) > 0).astype(int)
# Overwrite y_train with the new category DataFrame
y_train_filtered = category_df
y_val_filtered = category_df_val

print(y_train_filtered.sum()/len(y_train))

Service_Colonoscopy                0.038356
Service_EGD                        0.015624
Service_Gallbladder_surgery        0.002674
Service_CT_scan                    0.045394
Service_Infusions_drugs            0.160673
Service_Fentanyl                   0.024984
Service_Orthopedic_surgery         0.023788
Service_ENT_Respiratory_surgery    0.009923
Service_Electrocardiogram          0.115701
Service_Hospital_Observation       0.049968
dtype: float64


As we can see, the prevalence of each service is very low compared to the overall population

## Train the model
Each model is trained on the same input diagnoses because every member could potentially receive any service, but we train separate models because each service is driven by different patterns and therefore needs its own prediction process.



In [38]:


@contextmanager
def tqdm_joblib(tqdm_object):
    """
    Context manager to patch joblib to report into tqdm progress bar.
    """
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)

        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

In [39]:


# Train a separate model for each service, given the same diagnoses
def train_ovr(model_factory, X, Y, desc="Training OVR"):
    n_labels = Y.shape[1]
    models = []
    for i in tqdm(range(n_labels), desc=desc):
        y = Y.iloc[:, i].values if isinstance(Y, pd.DataFrame) else Y[:, i]
        clf = model_factory()
        clf.fit(X, y)
        models.append(clf)
    return models

# Helper: predict positive class probabilities for OVR models 
def predict_proba_ovr(models, X):
    probs = []
    for clf in models:
        proba = clf.predict_proba(X)
        # take P(y=1)
        if proba.ndim == 2 and proba.shape[1] >= 2:
            probs.append(proba[:, 1])
        else:
            probs.append(np.ravel(proba))  # fallback if a single column is returned
    return np.vstack(probs).T  # shape: (n_samples, n_labels)

# Return predictions after running K-fold cross-validation with k=3
def oof_probs_ovr(model_factory, X, Y, n_splits=3, random_state=42):
    X_np = X.values if isinstance(X, pd.DataFrame) else X
    Y_df = Y if isinstance(Y, pd.DataFrame) else pd.DataFrame(Y)
    n_samples, n_labels = X_np.shape[0], Y_df.shape[1]
    oof = np.zeros((n_samples, n_labels), dtype=float)

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    for fold, (tr_idx, va_idx) in enumerate(kf.split(X_np), start=1):
        X_tr, X_va = X_np[tr_idx], X_np[va_idx]
        for j in tqdm(range(n_labels), desc=f"CV fold {fold}"):
            y_tr = Y_df.iloc[tr_idx, j].values
            # Handle degenerate fold (only one class)
            if np.unique(y_tr).size < 2:
                # Use the prevalence in the training fold as a naive probability
                p = float(y_tr.mean())
                oof[va_idx, j] = p
                continue
            clf = model_factory()
            clf.fit(X_tr, y_tr)
            proba = clf.predict_proba(X_va)
            proba_pos = proba[:, 1] if proba.ndim == 2 and proba.shape[1] >= 2 else np.ravel(proba)
            oof[va_idx, j] = proba_pos
    return oof

X_val = None
y_val = None
has_holdout = False



# Tune XGB hyperparameters using micro-AUPRC and RandomizedSearchCV
# Some classes are much more rare than others, so it is important to use micro-AUPRC rather than macro-AUPRC
def tune_xgb_hyperparams(X, Y, n_iter=25, random_state=42):

    base_est = XGBClassifier(
        objective='binary:logistic',
        eval_metric='logloss',
        tree_method='hist',
        random_state=random_state,
        n_jobs=-1
    )

    ovr = OneVsRestClassifier(base_est)
    micro_auprc_scorer = make_scorer(average_precision_score, needs_proba=True, average='micro')

    param_dist = {
        'estimator__n_estimators': [200, 300, 400, 600, 800],
        'estimator__learning_rate': [0.01, 0.03, 0.05, 0.07, 0.1],
        'estimator__max_depth': [3, 4, 5, 6, 8],
        'estimator__min_child_weight': [1, 2, 3, 5, 7, 10],
        'estimator__subsample': [0.6, 0.8, 1.0],
        'estimator__colsample_bytree': [0.6, 0.8, 1.0],
        'estimator__gamma': [0, 0.5, 1, 2],
        'estimator__reg_alpha': [0, 1e-3, 1e-2, 1e-1, 1],
        'estimator__reg_lambda': [0.5, 1, 2, 5, 10],
    }

    cv = KFold(n_splits=3, shuffle=True, random_state=random_state)

    search = RandomizedSearchCV(
        estimator=ovr,
        param_distributions=param_dist,
        n_iter=n_iter,
        scoring=micro_auprc_scorer,
        cv=cv,
        verbose=0,      # tqdm will handle progress display
        n_jobs=-1,
        random_state=random_state,
        refit=True
    )

    Y_fit = Y.values if isinstance(Y, pd.DataFrame) else Y

    # Each candidate is evaluated across `cv.get_n_splits(X)` splits
    total_tasks = n_iter * cv.get_n_splits(X)
    with tqdm_joblib(tqdm(total=total_tasks, desc="Hyperparameter search (CV tasks)")):
        search.fit(X, Y_fit)

    # Extract best XGB params
    best_params = {k.replace('estimator__', ''): v
                   for k, v in search.best_params_.items()
                   if k.startswith('estimator__')}
    return best_params, search.best_score_

print("\n=== Hyperparameter tuning (XGBoost, micro-AUPRC CV) ===")
best_xgb_params, best_cv_score = tune_xgb_hyperparams(X_train_filtered, y_train_filtered, n_iter=25, random_state=42)
print("Best CV micro-AUPRC:", f"{best_cv_score:.6f}")
print("Best XGB params:", best_xgb_params)

# Model factory using the tuned hyperparameters 
def xgb_factory():
    return XGBClassifier(
        objective='binary:logistic',
        eval_metric='logloss',
        tree_method='hist',  # use 'gpu_hist' if you have a GPU
        random_state=42,
        n_jobs=-1,
        **best_xgb_params
    )

# Train & evaluate tuned XGBoost 
print(f"\n=== XGBoost (tuned) ===")
if has_holdout:
    # Train OVR models on train, evaluate on holdout
    models = train_ovr(xgb_factory, X_train_filtered, y_train_filtered, desc=f"Training OVR (XGBoost tuned)")
    y_scores = predict_proba_ovr(models, X_val)
    # Make sure y_val is array-like (n_samples, n_labels)
    y_true = y_val.values if isinstance(y_val, pd.DataFrame) else y_val
    micro_auprc = average_precision_score(y_true, y_scores, average='micro')
else:
    # CV fallback (OOF micro-AUPRC)
    y_oof = oof_probs_ovr(xgb_factory, X_train_filtered, y_train_filtered, n_splits=3, random_state=42)
    y_true = y_train_filtered.values if isinstance(y_train_filtered, pd.DataFrame) else y_train_filtered
    micro_auprc = average_precision_score(y_true, y_oof, average='micro')

print(f"XGBoost (tuned) micro-AUPRC: {micro_auprc:.6f}")

# --- Summary ---
print("\n--- Summary (micro-AUPRC) ---")
print(f"XGBoost (tuned): {micro_auprc:.6f}")


=== Hyperparameter tuning (XGBoost, micro-AUPRC CV) ===


Hyperparameter search (CV tasks):   0%|          | 0/75 [00:00<?, ?it/s]

Best CV micro-AUPRC: 0.324150
Best XGB params: {'subsample': 0.6, 'reg_lambda': 5, 'reg_alpha': 0.1, 'n_estimators': 600, 'min_child_weight': 1, 'max_depth': 5, 'learning_rate': 0.03, 'gamma': 2, 'colsample_bytree': 0.6}

=== XGBoost (tuned) ===


CV fold 1:   0%|          | 0/10 [00:00<?, ?it/s]

CV fold 2:   0%|          | 0/10 [00:00<?, ?it/s]

CV fold 3:   0%|          | 0/10 [00:00<?, ?it/s]

XGBoost (tuned) micro-AUPRC: 0.324640

--- Summary (micro-AUPRC) ---
XGBoost (tuned): 0.324640


This micro-AUPRC is much higher than the average prevalence, so our model is much better than random

## Save the models, and load them back in

In [40]:




# Create a list to hold trained models
trained_models = []

# Show progress bar while training each label
for i in tqdm(range(y_train_filtered.shape[1]), desc="Training models"):
    model = xgb_factory()
    model.fit(X_train_filtered, y_train_filtered.iloc[:, i])

    trained_models.append(model)



Training models:   0%|          | 0/10 [00:00<?, ?it/s]

In [41]:


# Create a directory to store models
os.makedirs("saved_models", exist_ok=True)

# Save each model individually
for i, model in enumerate(trained_models):
    joblib.dump(model, f"saved_models/xgb_model_{i}.pkl")


In [42]:
# Load models back into a list
loaded_models = []
for i in range(y_train_filtered.shape[1]):
    print(i)
    model = joblib.load(f"saved_models/xgb_model_{i}.pkl")
    loaded_models.append(model)


    
print("✅ All models loaded correctly and produce identical predictions.")


0
1
2
3
4
5
6
7
8
9
✅ All models loaded correctly and produce identical predictions.


## Compute accuracy, recall, and count for each model

In [43]:

results = []

for i, model in enumerate(loaded_models):
    print(i)
    # Get true labels for this target
    y_true = y_val_filtered.iloc[:, i] if hasattr(y_val_filtered, "iloc") else y_val_filtered[:, i]
    
    # Predict probabilities or raw predictions
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_val_filtered)[:, 1]  # positive class
    else:
        y_proba = model.predict(X_val_filtered)
    
    # Apply threshold
    y_pred = (y_proba >= 0.5).astype(int)
    
    # Compute metrics
    acc = accuracy_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred, zero_division=0)
    
    results.append({
        "target": y_val_filtered.columns[i] if hasattr(y_val_filtered, "columns") else f"target_{i}",
        "accuracy": acc,
        "recall": rec,
        "n": sum(y_true)
    })

# Create DataFrame and sort
metrics_df = pd.DataFrame(results)
metrics_df = metrics_df.sort_values(by=["accuracy", "recall"], ascending=False).reset_index(drop=True)

print(metrics_df)


0
1
2
3
4
5
6
7
8
9
                            target  accuracy    recall    n
0      Service_Gallbladder_surgery  0.996622  0.333333   12
1  Service_ENT_Respiratory_surgery  0.991273  0.074074   27
2                      Service_EGD  0.985642  0.303571   56
3       Service_Orthopedic_surgery  0.976070  0.024096   83
4                 Service_Fentanyl  0.971847  0.040000  100
5              Service_Colonoscopy  0.966779  0.376712  146
6     Service_Hospital_Observation  0.954955  0.018868  159
7                  Service_CT_scan  0.950169  0.027933  179
8        Service_Electrocardiogram  0.883727  0.106888  421
9          Service_Infusions_drugs  0.829955  0.107143  616


In [44]:
metrics_df[metrics_df['n'] > 10]

,target,accuracy,recall,n
0,Service_Gallbladder_surgery,0.996622,0.333333,12
1,Service_ENT_Respiratory_surgery,0.991273,0.074074,27
2,Service_EGD,0.985642,0.303571,56
3,Service_Orthopedic_surgery,0.976070,0.024096,83
4,Service_Fentanyl,0.971847,0.040000,100
5,Service_Colonoscopy,0.966779,0.376712,146
6,Service_Hospital_Observation,0.954955,0.018868,159
7,Service_CT_scan,0.950169,0.027933,179
8,Service_Electrocardiogram,0.883727,0.106888,421
9,Service_Infusions_drugs,0.829955,0.107143,616


In [45]:
recall_low = metrics_df.sort_values(by=["recall", "n"], ascending=True).reset_index(drop=True)
recall_low[(recall_low['n'] > 10) & (recall_low['recall'] == 0)] 


,target,accuracy,recall,n


It is a given that with only the data we have, it is difficult to fully predict whether a patient would need a service. However, our recall is much higher than a random guess. Our goal is not to predict the necessity of a service perfectly, but rather warn patients who have an increased risk of needing certain services, and this does a great job at doing that for a number of services. None of the services we are predicting have a recall of 0 with a count > 10

## Gather more evaluation metrics

In [46]:


y_true_list = []
y_score_list = []
y_pred_list = []

results = []

# To store FP score series per target
fp_scores_by_target = {}
# Also store a quick textual snapshot per label
fp_top_scores_by_target = {}

# Use the index from y_val_filtered if available (for traceability of FP cases)
if hasattr('y_val_filtered', '__iter__') and hasattr(y_val_filtered, 'index'):
    sample_index = y_val_filtered.index
else:
    # Fallback: simple integer index
    # We'll define it inside the loop when we know the number of rows
    sample_index = None

for i, model in enumerate(loaded_models):
    # True labels for this target
    y_true = y_val_filtered.iloc[:, i] if hasattr(y_val_filtered, "iloc") else y_val_filtered[:, i]
    y_true_arr = np.asarray(y_true).ravel()

    # Predicted scores and threshold choice
    if hasattr(model, "predict_proba"):
        y_score = model.predict_proba(X_val_filtered)[:, 1]
        threshold = 0.5
    elif hasattr(model, "decision_function"):
        y_score = model.decision_function(X_val_filtered)
        threshold = 0.0  # decision_function is typically centered at 0
    else:
        # Fallback: uses 0/1 predictions as score
        y_score = model.predict(X_val_filtered)
        threshold = 0.5

    y_score_arr = np.asarray(y_score).ravel()

    # Build an index if not present yet
    if sample_index is None:
        sample_index = pd.RangeIndex(start=0, stop=len(y_score_arr), step=1)

    # Thresholded predictions (for acc/rec and false positives)
    y_pred = (y_score_arr >= threshold).astype(int)
    y_pred_list.append(y_pred)

    # Accuracy and recall
    acc = accuracy_score(y_true_arr, y_pred)
    rec = recall_score(y_true_arr, y_pred, zero_division=0)

    #False positives (pred=1, true=0) 
    fp_mask = (y_pred == 1) & (y_true_arr == 0)
    fp_scores = y_score_arr[fp_mask]

    # Store FP scores as a Series for easy analysis later
    label_name = y_val_filtered.columns[i] if hasattr(y_val_filtered, "columns") else f"target_{i}"
    fp_scores_series = pd.Series(fp_scores, index=np.array(sample_index)[fp_mask], name=f"{label_name}_fp_score")
    fp_scores_by_target[label_name] = fp_scores_series

    # Small textual summary (top-5 highest FP scores)
    if fp_scores.size > 0:
        # Sort descending by score for top offenders
        fp_sorted_idx = np.array(sample_index)[fp_mask][np.argsort(-fp_scores)]
        top_k = min(5, fp_scores.size)
        top_scores = fp_scores[np.argsort(-fp_scores)[:top_k]]
        top_ids = fp_sorted_idx[:top_k]
        fp_top_scores_by_target[label_name] = list(zip(top_ids, top_scores))
        fp_mean = float(np.mean(fp_scores))
        fp_median = float(np.median(fp_scores))
    else:
        fp_top_scores_by_target[label_name] = []
        fp_mean = np.nan
        fp_median = np.nan

    # True positives
    true_positives = int(np.sum((y_pred == 1) & (y_true_arr == 1))) 
    false_positives = int(np.sum((y_pred == 1) & (y_true_arr == 0)))
    prevalence = float(np.sum(y_true_arr == 1)) / float(len(y_true_arr))
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0.0
    
    results.append({
        "target": label_name,
        "accuracy": acc,
        "recall": rec,
        "n_pos": int(np.sum(y_true_arr)),
        "prevalence":prevalence,
        "true_positives": true_positives,  # <-- Added field
        "false_positives": int(fp_mask.sum()),
        "precision":precision,
        "fp_score_mean": fp_mean,
        "fp_score_median": fp_median
    })

    y_true_list.append(y_true_arr)
    y_score_list.append(y_score_arr)

# Stack into (n_samples, n_targets)
Y_true = np.column_stack(y_true_list)
Y_score = np.column_stack(y_score_list)

# Micro AUPRC (global, preferred) 
micro_auprc = average_precision_score(Y_true, Y_score, average='micro')

# Macro and Weighted AUPRC for comparison:
macro_auprc = average_precision_score(Y_true, Y_score, average='macro')       # equal weight per target
weighted_auprc = average_precision_score(Y_true, Y_score, average='weighted') # weighted by positives in each target

# Per-target AP
ap_per_target = average_precision_score(Y_true, Y_score, average=None)

metrics_df = pd.DataFrame(results).sort_values(by=["accuracy", "recall"], ascending=False).reset_index(drop=True)

# Align AP to the current sorted order index
try:
    metrics_df["AP"] = ap_per_target[metrics_df.index]
except Exception:
    # Safer alignment by label name (in case of shape mismatch)
    label_names = [y_val_filtered.columns[i] if hasattr(y_val_filtered, "columns") else f"target_{i}" for i in range(Y_true.shape[1])]
    ap_map = {label_names[i]: ap_per_target[i] for i in range(len(label_names))}
    metrics_df["AP"] = metrics_df["target"].map(ap_map)

# Print metrics
print("Micro AUPRC:", micro_auprc)
print("Macro AUPRC:", macro_auprc)
print("Weighted AUPRC (by positives):", weighted_auprc)
print("\nPer-Label Metrics (incl. false positives):")






Micro AUPRC: 0.334415626058403
Macro AUPRC: 0.30130457481599
Weighted AUPRC (by positives): 0.32365158836687086

Per-Label Metrics (incl. false positives):


In [47]:
metrics_df

,target,accuracy,recall,n_pos,prevalence,true_positives,false_positives,precision,fp_score_mean,fp_score_median,AP
0,Service_Gallbladder_surgery,0.996622,0.333333,12,0.003378,4,4,0.500000,0.559908,0.559758,0.571339
1,Service_ENT_Respiratory_surgery,0.991273,0.074074,27,0.007601,2,6,0.250000,0.606400,0.580063,0.398081
2,Service_EGD,0.985642,0.303571,56,0.015766,17,12,0.586207,0.662475,0.640888,0.483883
3,Service_Orthopedic_surgery,0.976070,0.024096,83,0.023367,2,4,0.333333,0.551802,0.531656,0.232641
4,Service_Fentanyl,0.971847,0.040000,100,0.028153,4,4,0.500000,0.554395,0.542135,0.364427
5,Service_Colonoscopy,0.966779,0.376712,146,0.041104,55,27,0.670732,0.738224,0.745542,0.163132
6,Service_Hospital_Observation,0.954955,0.018868,159,0.044764,3,4,0.428571,0.604274,0.539807,0.136945
7,Service_CT_scan,0.950169,0.027933,179,0.050394,5,3,0.625000,0.695375,0.712577,0.162892
8,Service_Electrocardiogram,0.883727,0.106888,421,0.118525,45,37,0.548780,0.619662,0.553613,0.355244
9,Service_Infusions_drugs,0.829955,0.107143,616,0.173423,66,54,0.550000,0.619017,0.602906,0.144464


Given that the prevalence of each service is low, the precision, AUPRC and AP scores show that the model is much better at detecting services than a random guess. Even if some patients do not end up needing the service, it is worth it to have a few extra members get a checkup rather than having a member suffer a serious injury that could have been prevented. 


# Association Rule Mining: X -> Y
Association rule mining helps us map diagnoses to services in plain English, creating rules that others can understand

In [48]:



# Configuration 
MIN_SUPPORT = 0.01          # min fraction of samples for an itemset
MIN_CONFIDENCE = 0.10       
TOP_K_RULES = 100             # rules to print per label
MAX_ANTECEDENT = 2        
MAX_BINS = 3              
MIN_UNIQUE_NUMERIC = 5      # treat as numeric (else binary-like)
MIN_ITEM_FREQ = 0.01        # drop very rare feature items 


# Combine all X and Y since we don't need train/val splits

X_ARM = pd.concat([X_train_filtered, X_val_filtered])
Y_ARM = pd.concat([y_train_filtered, y_val_filtered])


# Convert to DataFrame if needed
X_ARM = pd.DataFrame(X_ARM) if not isinstance(X_ARM, pd.DataFrame) else X_ARM
Y_ARM = pd.DataFrame(Y_ARM) if not isinstance(Y_ARM, pd.DataFrame) else Y_ARM

label_names = list(Y_ARM.columns)


# Binarize X into boolean "items" -- either through bins or one-hot encoding
def binarize_X_for_arm(X, max_bins=3, min_unique=5, min_item_freq=0.01):
    items = {}
    for col in X.columns:
        s = X[col]
        if pd.api.types.is_numeric_dtype(s):
            # Binary-like numeric
            non_na = s.dropna()
            if non_na.nunique() <= 2 or len(non_na) < 10:
                # Treat non-zero as True
                mask = (s.fillna(0) != 0)
                if mask.mean() >= min_item_freq:
                    items[f"X::{col}=1"] = mask.astype(bool)
            else:
                # Discretize into quantile bins
                try:
                    bins = min(max_bins, int(np.clip(non_na.nunique(), 2, max_bins)))
                    cats = pd.qcut(s, q=bins, duplicates='drop')
                    dummies = pd.get_dummies(cats, prefix=f"X::{col}", dummy_na=False)
                    for dcol in dummies.columns:
                        mask = dummies[dcol].astype(bool)
                        if mask.mean() >= min_item_freq:
                            items[str(dcol)] = mask
                except Exception:
                    # Fallback median split
                    thr = float(non_na.median())
                    le_mask = (s <= thr).fillna(False)
                    gt_mask = (s > thr).fillna(False)
                    if le_mask.mean() >= min_item_freq:
                        items[f"X::{col}<= {thr:.4g}"] = le_mask
                    if gt_mask.mean() >= min_item_freq:
                        items[f"X::{col}> {thr:.4g}"] = gt_mask
        else:
            # Categorical -> one-hot
            cats = s.astype('category')
            dummies = pd.get_dummies(cats, prefix=f"X::{col}", dummy_na=False)
            for dcol in dummies.columns:
                mask = dummies[dcol].astype(bool)
                if mask.mean() >= min_item_freq:
                    items[str(dcol)] = mask
    if not items:
        return pd.DataFrame(index=X.index)
    return pd.DataFrame(items, index=X.index).astype(bool)

# Binarize Y (labels) into boolean columns with Y:: prefix 
def binarize_Y_for_arm(Y):
    Y_bool = Y.astype(bool)
    Y_bool.columns = [f"Y::{c}" for c in Y_bool.columns]
    return Y_bool.astype(bool)

X_items = binarize_X_for_arm(X_ARM, max_bins=MAX_BINS, min_unique=MIN_UNIQUE_NUMERIC, min_item_freq=MIN_ITEM_FREQ)
Y_items = binarize_Y_for_arm(Y_ARM)

if X_items.shape[1] == 0:
    print("[ARM] No feature items were created (all too rare or non-informative). Try lowering MIN_ITEM_FREQ or adjust binning.")
    rules_xy = pd.DataFrame(columns=["antecedents", "consequents", "support", "confidence", "lift"])
else:


    # Combine X and Y items
    XY = pd.concat([X_items, Y_items], axis=1)
    # Frequent itemsets
    try:
        freq = fpgrowth(XY, min_support=MIN_SUPPORT, use_colnames=True)
    except Exception:
        freq = apriori(XY, min_support=MIN_SUPPORT, use_colnames=True)
    # Calculate rules
    if freq.empty:
        rules_xy = pd.DataFrame(columns=["antecedents", "consequents", "support", "confidence", "lift"])
    else:
        rules = association_rules(freq, metric="confidence", min_threshold=MIN_CONFIDENCE)
        if rules.empty:
            rules_xy = pd.DataFrame(columns=["antecedents", "consequents", "support", "confidence", "lift"])
        else:
            # Convert frozensets, filter: consequents are one Y label; antecedents only X items
            def to_set(fs): return set(map(str, fs))
            rules["antecedents"] = rules["antecedents"].apply(to_set)
            rules["consequents"] = rules["consequents"].apply(to_set)
            rules = rules[
                (rules["consequents"].apply(len) == 1) &
                (rules["consequents"].apply(lambda s: all(x.startswith("Y::") for x in s))) &
                (rules["antecedents"].apply(len) >= 1) &
                (rules["antecedents"].apply(len) <= MAX_ANTECEDENT) &
                (rules["antecedents"].apply(lambda s: all(x.startswith("X::") for x in s)))
            ]
            if rules.empty:
                rules_xy = pd.DataFrame(columns=["antecedents", "consequents", "support", "confidence", "lift"])
            else:
                keep = [c for c in ["antecedents", "consequents", "support", "confidence", "lift", "leverage", "conviction"] if c in rules.columns]
                rules_xy = rules[keep].sort_values(
                    by=["confidence", "lift", "support"], ascending=[False, False, False]
                ).reset_index(drop=True)
    
# Formatting helpers
def _strip_prefix(name):
    # remove "X::" or "Y::" for readability
    if name.startswith("X::"):
        return name[3:]
    if name.startswith("Y::"):
        return name[3:]
    return name

def format_rules_for_label_xy(rules_df, label_name, top_k=100):
    """Return strings like: 'feat_bin1 & feat_bin2 ⇒ label (supp=..., conf=..., lift=...)'"""
    if rules_df is None or rules_df.empty:
        return ["(no rules found)"]
    y_token = f"Y::{label_name}"
    mask = rules_df["consequents"].apply(lambda s: s == {y_token})
    sub = rules_df[mask]
    if sub.empty:
        return ["(no rules found)"]
    lines = []
    for _, row in sub.head(top_k).iterrows():
        ants = " & ".join(sorted([_strip_prefix(a) for a in row["antecedents"]]))
        cons = _strip_prefix(next(iter(row["consequents"])))
        lines.append(f"{ants} ⇒ {cons} (supp={row['support']:.3f}, conf={row['confidence']:.3f}, lift={row.get('lift', np.nan):.3f})")
    return lines

# Plain-text per-label output 
print("\n=====================")
print("X → Y Association Rules (confidence ≥ 0.10)")
print("=====================")
for label in label_names:
    print(f"\n=== Target: {label} ===")
    lines = format_rules_for_label_xy(rules_xy, label, top_k=TOP_K_RULES)
    for line in lines:
        print(f"  - {line}")

# global snapshot
def print_global_rules_snapshot(rules_df, title, k=100):
    print(f"\n--- {title} (top {k}) ---")
    if rules_df is None or rules_df.empty:
        print("(no rules found)")
        return
    sub = rules_df.head(k)
    for _, row in sub.iterrows():
        ants = " & ".join(sorted([_strip_prefix(a) for a in row["antecedents"]]))
        cons = " & ".join(sorted([_strip_prefix(c) for c in row["consequents"]]))
        print(f"{ants} ⇒ {cons} (supp={row['support']:.3f}, conf={row['confidence']:.3f}, lift={row.get('lift', np.nan):.3f})")

print_global_rules_snapshot(rules_xy, "Global X → Y rules", k=100)


X → Y Association Rules (confidence ≥ 0.10)

=== Target: Service_Colonoscopy ===
  - Diagnosis_K57.30=1 ⇒ Service_Colonoscopy (supp=0.011, conf=0.660, lift=16.961)

=== Target: Service_EGD ===
  - (no rules found)

=== Target: Service_Gallbladder_surgery ===
  - (no rules found)

=== Target: Service_CT_scan ===
  - (no rules found)

=== Target: Service_Infusions_drugs ===
  - Diagnosis_Z79.899=1 ⇒ Service_Infusions_drugs (supp=0.029, conf=0.369, lift=2.259)
  - Diagnosis_E66.9=1 ⇒ Service_Infusions_drugs (supp=0.011, conf=0.300, lift=1.839)
  - Diagnosis_K21.9=1 ⇒ Service_Infusions_drugs (supp=0.019, conf=0.299, lift=1.829)
  - Diagnosis_I10=1 ⇒ Service_Infusions_drugs (supp=0.031, conf=0.260, lift=1.591)
  - Diagnosis_E78.5=1 ⇒ Service_Infusions_drugs (supp=0.015, conf=0.231, lift=1.418)
  - Diagnosis_E55.9=1 ⇒ Service_Infusions_drugs (supp=0.013, conf=0.226, lift=1.386)
  - Diagnosis_F41.9=1 ⇒ Service_Infusions_drugs (supp=0.016, conf=0.217, lift=1.328)

=== Target: Service_Fentanyl

The first rule, Diagnosis_K57.30, indicates that members with "Diverticulosis of large intestine without perforation or abscess without bleeding" are more likely to get a colonoscopy than other members. This potentially indicates that members who have this diagnosis should look into getting a colonoscopy if they have not done so already

## Importance scores on specific services

In [49]:


# Assume the target name and index
target_name = "Service_Gallbladder_surgery"

# Find the index of the target in y_val_filtered
target_index = y_val_filtered.columns.get_loc(target_name)

# Get the corresponding model
model = loaded_models[target_index]

# Get feature names from X_val_filtered
feature_names = X_val_filtered.columns

# Determine feature importance based on model type
if hasattr(model, "coef_"):
    # For linear models (e.g., Logistic Regression)
    importance = model.coef_.ravel()
elif hasattr(model, "feature_importances_"):
    # For tree-based models (e.g., RandomForest, XGBoost)
    importance = model.feature_importances_
else:
    raise ValueError("Model does not support feature importance extraction.")

# Create a DataFrame of feature importances
importance_df = pd.DataFrame({
    "feature": feature_names,
    "importance": importance
}).sort_values(by="importance", ascending=False)

# Display top 20 most important features
print("Top 20 important features for predicting", target_name)
print(importance_df.head(20))

Top 20 important features for predicting Service_Gallbladder_surgery
               feature  importance
239   Diagnosis_K80.10    0.157337
240   Diagnosis_K80.20    0.116253
49     Diagnosis_D25.9    0.047053
236    Diagnosis_K76.0    0.042865
370  Diagnosis_Z79.899    0.041298
219    Diagnosis_K42.9    0.038930
85     Diagnosis_E78.2    0.038891
214    Diagnosis_K21.9    0.037684
168    Diagnosis_H52.4    0.037038
173      Diagnosis_I10    0.036944
407   Diagnosis_Z90.49    0.036007
83    Diagnosis_E78.00    0.035318
119    Diagnosis_F41.1    0.034203
349    Diagnosis_N92.0    0.034005
158  Diagnosis_H04.123    0.033700
54     Diagnosis_D50.9    0.030841
87     Diagnosis_E78.5    0.030042
1      Diagnosis_238.2    0.029270
147   Diagnosis_G47.00    0.029254
78    Diagnosis_E66.01    0.029149


This indicates that members with gallstones are more likely to get gallbladder surgery, and if they have not received one, should likely be warned about it